In [1]:
import os

In [2]:
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%pwd

'e:\\projects\\DL\\Chicken-Disease-Classification\\research'

In [22]:
os.chdir("../")

In [23]:
%pwd

'e:\\projects\\DL\\Chicken-Disease-Classification'

In [24]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [25]:
# ! pip install python-box
# ! pip install pyYAML
# ! pip install ensure==1.0.2


In [26]:
import os
print("[DEBUG] Current Working Directory:", os.getcwd())
print("[DEBUG] File exists?:", os.path.exists("config/config.yaml"))

[DEBUG] Current Working Directory: e:\projects\DL\Chicken-Disease-Classification
[DEBUG] File exists?: True


In [27]:
from CnnClassifier.constants import *
from CnnClassifier.utils.common import read_yaml, create_directories

In [41]:
class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.paramns = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

In [29]:
import os
import urllib.request as request
import zipfile
from CnnClassifier import logger
from CnnClassifier.utils.common import get_size

In [44]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f'{filename} downloaded! with following info: \n{headers}')
        else:
            logger.info(f'file already exists of size: {get_size(Path(self.config.local_data_file))}')


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [45]:
# pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-20 01:09:19,696:  INFO:  common:  yaml file: config\config.yaml loaded successully]
[2025-06-20 01:09:19,705:  INFO:  common:  yaml file: params.yaml loaded successully]
[2025-06-20 01:09:19,709:  INFO:  common:  created directory at: artifacts]
[2025-06-20 01:09:19,713:  INFO:  common:  created directory at: artifacts/data_ingestion]
[2025-06-20 01:09:19,722:  INFO:  3167276667:  file already exists of size: ~ 11345 KB]


In [38]:
# import yaml
# with open("config/config.yaml") as f:
#     content = yaml.safe_load(f)
# print(content)